## Eficiência Operacional

### 🎯Objetivo dessa secção

Avaliar como se comporta as vendas e o crescimento delas.

As principais perguntas a serem respondidas são:  
✅ Existem meses com vendas **consistentemente maiores**?  
✅ O padrão muda de um ano para outro?  
✅ O **lucro acompanha o crescimento das vendas** ou não?  

In [1]:
import pandas as pd

In [2]:
finances = pd.read_excel('../../database/financial.xlsx')
finances.head()

,segment,country,product,discount_band,units_sold,manufacturing_price,sale_price,gross_sales,discounts,sales,cogs,profit,date,month_number,month_name,year
0,Government,Canada,Montana,High,200,5,350,70000.0,9800.0,60200.0,52000.0,8200.0,2014-05-01,5,May,2014
1,Small Business,Germany,Carretera,Low,214,3,300,64200.0,1284.0,62916.0,53500.0,9416.0,2013-10-01,10,October,2013
2,Small Business,Germany,VTT,Low,214,250,300,64200.0,1284.0,62916.0,53500.0,9416.0,2013-10-01,10,October,2013
3,Midmarket,Canada,Paseo,Low,218,10,15,3270.0,130.8,3139.2,2180.0,959.2,2014-09-01,9,September,2014
4,Government,Germany,Paseo,High,241,10,20,4820.0,482.0,4338.0,2410.0,1928.0,2014-10-01,10,October,2014


In [4]:
# Existem meses com vendas consistentemente maiores?

# Agregação mensal de todos os anos
monthly_seasonality = (
    finances
        .groupby("month_name")[["sales", "profit", "units_sold"]]
        .sum()
        .reset_index()
)

monthly_seasonality.sort_values("sales", ascending=False)

,month_name,sales,profit,units_sold
10,October,21671431.02,3439781.02,201104
2,December,17367228.98,2717329.98,155306
9,November,12651417.50,1370102.50,121131
11,September,10882697.27,1786735.27,107881
6,June,9518893.82,1473753.82,103302
5,July,8102920.19,923865.69,69343
3,February,7297531.39,1148547.39,55115
0,April,6964775.08,929984.58,78880
4,January,6607761.69,814028.69,67830
8,May,6210211.06,828640.06,51771


In [5]:
# ordenar os meses de forma correta

month_order = [
    "January", "February", "March", "April", "May", "June",
    "July", "August", "September", "October", "November", "December"
]

monthly_seasonality["month_name"] = pd.Categorical(
    monthly_seasonality["month_name"],
    categories=month_order,
    ordered=True
)

monthly_seasonality.sort_values("month_name")


,month_name,sales,profit,units_sold
4,January,6607761.69,814028.69,67830
3,February,7297531.39,1148547.39,55115
7,March,5586859.87,669866.87,53420
0,April,6964775.08,929984.58,78880
8,May,6210211.06,828640.06,51771
6,June,9518893.82,1473753.82,103302
5,July,8102920.19,923865.69,69343
1,August,5864622.42,791066.42,60705
11,September,10882697.27,1786735.27,107881
10,October,21671431.02,3439781.02,201104


In [7]:
# O padrão muda de um ano para outro?

# Agregação por ano e mês
year_month_trend = (
    finances
        .groupby(["year", "month_name"])[["sales", "profit"]]
        .sum()
        .reset_index()
)

year_month_trend.sort_values(["year", "month_name"])


year_month_trend.pivot_table(
    index="month_name",
    columns="year",
    values="sales",
    aggfunc="sum"
)


year,2013,2014
month_name,,
April,NaN,6964775.08
August,NaN,5864622.42
December,5368441.08,11998787.90
February,NaN,7297531.39
January,NaN,6607761.69
July,NaN,8102920.19
June,NaN,9518893.82
March,NaN,5586859.87
May,NaN,6210211.06


In [8]:
# Tendência anual
yearly_trend = (
    finances
        .groupby("year")[["sales", "profit", "units_sold"]]
        .sum()
        .reset_index()
)

yearly_trend


,year,sales,profit,units_sold
0,2013,26415255.51,3878464.51,264674
1,2014,92311094.78,13015237.78,861114


In [9]:
yearly_trend_pct = yearly_trend.copy()

yearly_trend_pct[["sales_growth", "profit_growth"]] = (
    yearly_trend_pct[["sales", "profit"]]
        .pct_change()
)

yearly_trend_pct


,year,sales,profit,units_sold,sales_growth,profit_growth
0,2013,26415255.51,3878464.51,264674,NaN,NaN
1,2014,92311094.78,13015237.78,861114,2.494613,2.355771


## 📆 Sazonalidade e Tendência — Insights

### 📅 Meses com Vendas Consistentemente Maiores
**Outubro, dezembro e novembro** concentram os maiores volumes de vendas, caracterizando forte sazonalidade no segundo semestre.

---

### 🔄 O Padrão Muda de um Ano para Outro?
O padrão sazonal se mantém entre 2013 e 2014, com crescimento expressivo em praticamente todos os meses no segundo ano.

---

### 📈 O Lucro Acompanha o Crescimento das Vendas?
O lucro cresce junto com as vendas, porém em ritmo ligeiramente inferior, indicando **pressão sobre a margem** durante a expansão.

---

### 🧠 Conclusões da Etapa
A sazonalidade é clara e previsível. O crescimento do negócio exige maior controle de custos para preservar margens em períodos de alta demanda.